In [1]:
import matplotlib.pyplot as plt
#from PIL import Image
import matplotlib.image as mpimg
import numpy as np
#from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow.metrics
import pandas as pd
from sklearn.model_selection import train_test_split
import os, shutil, random
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler

/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/leona/anaconda3/envs/finalpro/lib

## Explore Metadata

In [2]:
meta = pd.read_csv('data/HAM10000_metadata.csv')

#### There is a class imbalance concerning diagnoses. We have to deal with that later on.

### Identify lesion_IDs with multiple images for the validation set
To ensure that we don't have the same images in the train and validation set

In [21]:
uni = meta.groupby('lesion_id').count()
uni = uni[uni['image_id'] == 1]
unique_list = list(uni.index)

In [6]:
# define function to identify if a lesion id has duplicate images
def is_duplicate(x):
    
    unique_list = list(uni.index)
    
    if x in unique_list:
        return 'no_duplicates'
    else:
        return 'has_duplicates'

In [23]:
meta['duplicate'] = meta['lesion_id']
meta['duplicate'] = meta['duplicate'].apply(is_duplicate)

In [24]:
no_dupl = meta[meta.duplicate == "no_duplicates"]

### Create a validation set with only non-duplicate IDs

In [11]:
y= no_dupl.dx

In [12]:
# apply the train-test-split. Here we only need Xtest. Stratified splitting ensures equal distribution of classes
_, val_data = train_test_split(no_dupl, random_state=43, stratify=y, test_size=0.20)

In [13]:
val_data['dx'].value_counts()

nv       883
bkl       88
mel       46
bcc       35
akiec     30
vasc      13
df         8
Name: dx, dtype: int64

### Remove validation data from the training data

In [15]:
train_data = pd.concat([meta, val_data]).drop_duplicates(keep=False)

In [16]:
train_data.shape

(8912, 8)

### Create folder structure for training and validation data and load images

In [17]:
train_data = train_data.set_index("image_id")
val_data = val_data.set_index("image_id")

train_id = list(train_data.index)
test_id = list(val_data.index)

In [18]:
source_dir = "/Users/leona/PythonWork/Github_Projects/Final_Pro/data/ISIC2018_Task3_Training_Input/"

In [19]:

# Create train and validation directory
train_dir = os.path.join(source_dir, "train")
dirs.append(train_dir)
validation_dir = os.path.join(source_dir, "validation")
dirs.append(validation_dir)


### Rescale and Reshape all images (original + augmented)

### Setup model input

In [25]:
num_train_samples = len(train_data)
num_val_samples = len(val_data)
train_batch_size = 10
val_batch_size = 10
image_size = 224

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

## Transfer learning with pre-trained models

### Setup MobileNet

In [ ]:
datagen=ImageDataGenerator(preprocessing_function= tensorflow.keras.applications.mobilenet.preprocess_input)
  
def my_generator():
    train_batch=datagen.flow_from_dataframe(copy_train,
                                        x_col="full_id",
                                        y_col="dx", class_mode="categorical", target_size=(image_size,image_size), batch_size=train_batch_size)
    batch_2= datagen.flow_from_dataframe(copy_train,x_col="full_id",
                                        y_col="age",class_mode="raw",
                                     target_size=(image_size,image_size), batch_size=train_batch_size)
    
    while True:
        print(batch_2[0])


In [270]:
def createGenerator(df):

    while True:
        # suffled indices    
        idx = df.sample(n=10, replace=False).index
        # create image generator
        datagen = ImageDataGenerator(
            preprocessing_function= tensorflow.keras.applications.mobilenet.preprocess_input)

        batches = datagen.flow_from_dataframe(df.loc[idx], x_col="full_id",y_col="dx", shuffle=False)
        for batch in batches:
        
            yield [batch[0], df.loc[idx[["age", "sex"]].values]], batch[1]

In [210]:
train_batch = combined_generator(copy_train)

In [177]:
copy_train.set_index("aug_id", inplace=True)

In [183]:
full_id = []
for picture in copy_train.index:
    label = copy_train.loc[picture,'dx']
    full_id.append(train_dir + label + "/" + picture)

In [185]:
copy_train["full_id"] = full_id

/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [213]:
datagen = ImageDataGenerator(
    preprocessing_function= tensorflow.keras.applications.mobilenet.preprocess_input)

train_batches = datagen.flow_from_directory(train_dir,
                                            target_size=(image_size,image_size),
                                            batch_size=train_batch_size)

valid_batches = datagen.flow_from_directory(validation_dir,
                                            target_size=(image_size,image_size),
                                            batch_size=val_batch_size)

# Note: shuffle=False causes the test dataset to not be shuffled
test_batches = datagen.flow_from_directory(validation_dir,
                                            target_size=(image_size,image_size),
                                            batch_size=1,
                                            shuffle=False)

Found 38890 images belonging to 7 classes.
Found 1103 images belonging to 7 classes.
Found 1103 images belonging to 7 classes.


In [156]:
# We need to choose how many layers we actually want to be trained.

# Here we are freezing the weights of all layers except the
# last 23 layers in the new model.
# The last 23 layers of the model will be trained.

for layer in model.layers[:-23]:
    layer.trainable = False

# Merge CNN output with demographic data 

### Load demographic features for all images (augmented images included)

In [58]:
diagnose = list(meta['dx'].unique())
train_dir = "/Users/leona/PythonWork/Github_Projects/Final_Pro/data/ISIC2018_Task3_Training_Input/train/"
validation_dir = "/Users/leona/PythonWork/Github_Projects/Final_Pro/data/ISIC2018_Task3_Training_Input/validation/"

In [28]:
aug_list = []
for d in diagnose:
    aug_list.append(os.listdir(train_dir + d))

In [59]:
val_list = []
for d in diagnose:
    val_list.append(os.listdir(validation_dir + d))

In [60]:
aug_list = [item for sublist in aug_list for item in sublist]
val_list = [item for sublist in val_list for item in sublist]

In [29]:
img_id = list(meta.image_id)
meta['img_nr'] = [image[5:] for image in meta.image_id]

In [31]:
# Create helper function to extract pure image_id's
def extract_id(x):
        
    if "ISIC" in x:
        return x[5:12]
    else:
        return  x[:7]

In [32]:
aug_img = pd.DataFrame(aug_list, columns=['aug_id'])
aug_img['img_nr'] = aug_img['aug_id'].apply(extract_id)

In [64]:
val_img = pd.DataFrame(val_list, columns=['aug_id'])
val_img['img_nr'] = val_img['aug_id'].apply(extract_id)

In [45]:
complete_train = pd.merge(aug_img, meta, how='inner', on="img_nr", left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [65]:
complete_test = pd.merge(val_img, meta, how='inner', on="img_nr", left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

### Missing values, Min-Max-Scale and binary classification

In [140]:
copy_train = complete_train[['aug_id', 'dx', 'age', 'sex']]

In [141]:
copy_train['sex'] = copy_train['sex'].replace("unknown",copy_train['sex'].value_counts().index[0])

/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [143]:
copy_train['age'] = copy_train['age'].fillna(copy_train['age'].median())

/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [148]:
scaler = MinMaxScaler()
copy_train['age'] = scaler.fit_transform(copy_train['age'].values.reshape(-1,1))

/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [150]:
copy_train["sex"] = np.where(copy_train['sex']=="female",1,0)

/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [152]:
copy_test = complete_test[['aug_id','dx', 'age', 'sex']]
copy_test['sex'] = copy_test['sex'].replace("unknown",copy_train['sex'].value_counts().index[0])
copy_test['age'] = copy_test['age'].fillna(copy_train['age'].median())
scaler = MinMaxScaler()
copy_test['age'] = scaler.fit_transform(copy_test['age'].values.reshape(-1,1))
copy_test["sex"] = np.where(copy_test['sex']=="female",1,0)

/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

### Create concatenated model

In [190]:
mobile = tensorflow.keras.applications.mobilenet.MobileNet()

In [191]:
# CREATE THE MODEL ARCHITECTURE

# Exclude the last 5 layers of the above model.
# This will include all layers up to and including global_average_pooling2d_1
x = mobile.layers[-6].output

In [192]:
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)

In [193]:
model_dem = Model(inputs=mobile.input, outputs=predictions)

In [198]:
model_dem.load_weights("model.h5")

In [214]:
img_inpt = model_dem.layers[-3].output

In [249]:
test = copy_train.sample(n=10, replace=False)